<a href="https://colab.research.google.com/github/JuanJoMontilla/Senales-y-Sistemas/blob/main/Otros/Dashboard_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Descargar la base de datos
!gdown --id "1nkTCdDU7PzDVyDKWhoJ7cNN7rhdbjMw9"
!wget https://docs.google.com/spreadsheets/d/1nkTCdDU7PzDVyDKWhoJ7cNN7rhdbjMw9/edit?usp=sharing&ouid=118341214742540591942&rtpof=true&sd=true

In [ ]:
#Canción de prueba
!gdown --id "1I_THlsDbidYfHm3H2UmunHDBENT615me"
!wget https://drive.google.com/file/d/1I_THlsDbidYfHm3H2UmunHDBENT615me

In [ ]:
!unrar x canciones.rar

In [ ]:
!pip install yt-dlp
!apt-get install ffmpeg

In [ ]:
import os
import soundfile as sf
import numpy as np
import random
import subprocess

# Parámetros
segment_size = 5 * 48000  # 5 segundos de audio
segments_per_song = 4  # 4 segmentos por canción
genres = ['Clasica', 'Pop', 'Metal']  # Géneros musicales
sample_rate = 48000  # Asegurar que todas las canciones estén a esta frecuencia

# Descargar audio desde YouTube y convertir a formato adecuado
def download_audio_from_youtube(url, output_file='downloaded_song.wav'):
    try:
        # Descargar usando yt-dlp
        command = [
            'yt-dlp', '-x', '--audio-format', 'wav',
            '--output', 'temp_audio.%(ext)s', url
        ]
        subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Convertir audio a WAV, mono, frecuencia de muestreo de 48kHz
        command = [
            'ffmpeg', '-i', 'temp_audio.wav', '-ar', str(sample_rate), '-ac', '1',  # '1' es para audio mono
            output_file, '-y'
        ]
        subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Limpiar archivo temporal
        os.remove('temp_audio.wav')
        return output_file
    except Exception as e:
        print(f"Error descargando o procesando el audio de YouTube: {e}")
        return None

# Asegurar que el audio sea mono y tenga la longitud adecuada
def verify_audio_shape(song):
    if len(song.shape) > 1:  # Si es estéreo
        song = song.mean(axis=1)  # Convertir a mono
    return song

# Calcular el espectro de Fourier de los segmentos de una canción
def calculate_song_spectra(song, song_length):
    song_segments = []
    for _ in range(segments_per_song):
        segment_start = random.randint(0, song_length - segment_size)
        segment_end = segment_start + segment_size
        if segment_end > song_length:
            segment = np.zeros(segment_size)
            segment[:song_length - segment_start] = song[segment_start:]
        else:
            segment = song[segment_start:segment_end]
        spectrum = np.abs(np.fft.fft(segment))
        song_segments.append(spectrum)
    return song_segments

# Clasificación de la canción en uno de los géneros
def classify_song(song_path):
    try:
        # Leer la canción y asegurarse que tenga la frecuencia de muestreo correcta
        song, actual_sample_rate = sf.read(song_path)

        if actual_sample_rate != sample_rate:
            print(f"La frecuencia de muestreo del archivo es {actual_sample_rate}, debería ser {sample_rate}.")
            return None

        # Convertir la canción a mono si es estéreo
        song = verify_audio_shape(song)

        song_length = len(song)
        song_spectra = calculate_song_spectra(song, song_length)

        # Comparar el espectro de Fourier con la base de datos de géneros
        distances = {}
        for genre in genres:
            distances[genre] = np.mean([np.linalg.norm(spectrum - spectra_db[genre]) for spectrum in song_spectra])

        return min(distances, key=distances.get)
    except Exception as e:
        print(f"Error al clasificar la canción: {e}")
        return None

# Base de datos de espectros de Fourier por género
song_spectra = {}
for genre in genres:
    song_spectra[genre] = []
    genre_dir = os.path.join('canciones', genre)
    for song_path in os.listdir(genre_dir):
        song, _ = sf.read(os.path.join(genre_dir, song_path))
        song = verify_audio_shape(song)  # Asegurar que todas las canciones sean mono
        song_length = len(song)
        song_spectra[genre].extend(calculate_song_spectra(song, song_length))

spectra_db = {}
for genre in genres:
    spectra_db[genre] = np.mean(song_spectra[genre], axis=0)


# Graficar la separabilidad de los géneros y la canción clasificada
def plot_genre_separability(song_spectra, spectra_db, song_spectra_classified, genres):
    # Preparar datos para PCA
    X = []
    y = []
    genre_labels = []
    for genre in genres:
        # Tomar los espectros de cada género
        X.extend(song_spectra[genre])
        genre_labels.extend([genre] * len(song_spectra[genre]))

    # Normalizar los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Aplicar PCA para reducir a 2D
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)

    # Graficar los puntos de los géneros
    plt.figure(figsize=(10, 8))
    colors = sns.color_palette("viridis", len(genres))  # Colores con viridis como en tu imagen

    for i, genre in enumerate(genres):
        genre_data = X_pca[np.array(genre_labels) == genre]
        plt.scatter(genre_data[:, 0], genre_data[:, 1], label=genre, c=[colors[i]] * len(genre_data))

    # Calcular el PCA para la canción clasificada
    song_scaled = scaler.transform(song_spectra_classified.reshape(1, -1))
    song_pca = pca.transform(song_scaled)

    # Graficar el punto de la canción clasificada
    plt.scatter(song_pca[0, 0], song_pca[0, 1], color='red', marker='x', s=200, label='Canción analizada')

    # Añadir leyenda y etiquetas
    plt.title("Separabilidad de géneros y clasificación de la canción")
    plt.xlabel("PCA Componente 1")
    plt.ylabel("PCA Componente 2")
    plt.legend()
    plt.colorbar(plt.cm.ScalarMappable(cmap="viridis"), label="Escala de colores")
    plt.grid(True)
    plt.show()

def play_song_segment(song_path, segment_duration=5):
    try:
        song, sample_rate = sf.read(song_path)
        # Elegir un segmento aleatorio de la canción de duración definida
        segment_start = random.randint(0, len(song) - segment_duration * sample_rate)
        segment_end = segment_start + segment_duration * sample_rate
        song_segment = song[segment_start:segment_end]

        # Asegurarse que sea mono para la reproducción (si tiene más de un canal)
        if len(song_segment.shape) > 1:
            song_segment = song_segment.mean(axis=1)

        # Reproducir el segmento de audio
        return Audio(song_segment, rate=sample_rate)
    except Exception as e:
        print(f"Error al reproducir el segmento de la canción: {e}")
        return None

def classify_song_from_youtube(youtube_url):
    downloaded_song_path = download_audio_from_youtube(youtube_url)
    if downloaded_song_path:
        genre = classify_song(downloaded_song_path)
        if genre:
            print(f"La canción se clasifica como: {genre}")
            # Graficar la separabilidad
            song_spectra_classified = np.mean(calculate_song_spectra(sf.read(downloaded_song_path)[0], len(sf.read(downloaded_song_path)[0])), axis=0)
            plot_genre_separability(song_spectra, spectra_db, song_spectra_classified, genres)
            # Reproducir un segmento de la canción
            print("Reproduciendo un segmento de la canción analizada:")
            return play_song_segment(downloaded_song_path)
        else:
            print("No se pudo clasificar la canción.")
    else:
        print("No se pudo devolver la clasificación.")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns
from IPython.display import Audio
import soundfile as sf

# Ejecutar la clasificación y mostrar el gráfico
youtube_url = "https://www.youtube.com/watch?v=Qz9gmiLBVFA"  # Reemplazar con el enlace correcto
classify_song_from_youtube(youtube_url)

In [ ]:
import os
import soundfile as sf
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Parámetros
segment_size = 5 * 48000  # 5 segundos de audio
segments_per_song = 4  # 4 segmentos por canción
genres = ['Clasica', 'Pop', 'Metal']  # Géneros musicales
sample_rate = 48000

# Función para extraer segmentos de espectros de Fourier de las canciones
def calculate_song_spectra(song, song_length):
    song_segments = []
    for _ in range(segments_per_song):
        segment_start = np.random.randint(0, song_length - segment_size)
        segment_end = segment_start + segment_size
        if segment_end > song_length:
            segment = np.zeros(segment_size)
            segment[:song_length - segment_start] = song[segment_start:]
        else:
            segment = song[segment_start:segment_end]
        spectrum = np.abs(np.fft.fft(segment))
        song_segments.append(spectrum[:segment_size // 2])  # Usamos solo la mitad del espectro (frecuencias positivas)
    return song_segments

# Recolectar datos para entrenar
X = []
y = []

for genre in genres:
    genre_dir = os.path.join('canciones', genre)
    for song_path in os.listdir(genre_dir):
        song, _ = sf.read(os.path.join(genre_dir, song_path))
        song_length = len(song)
        if song_length >= segment_size:
            spectra = calculate_song_spectra(song, song_length)
            X.extend(spectra)
            y.extend([genre] * len(spectra))

# Convertir a formato numpy y aplanar cada espectro
X = np.array(X)
y = np.array(y)

# Aplanar los datos de 3D a 2D
X = X.reshape(X.shape[0], -1)

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy * 100:.2f}%")

# Guardar el modelo
joblib.dump(model, 'genre_classifier_model.pkl')
print("Modelo guardado como 'genre_classifier_model.pkl'")

In [ ]:
!pip install streamlit
!pip install pyngrok
!pip install yt-dlp
!pip install soundfile
!pip install joblib
!apt-get install ffmpeg

In [ ]:
%%writefile app.py
import streamlit as st
import soundfile as sf
import numpy as np
import subprocess
import joblib
from yt_dlp import YoutubeDL
import os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from IPython.display import Audio

# Parámetros
segment_size = 5 * 48000  # 5 segundos de audio, lo que resulta en 240,000 características en la FFT
sample_rate = 48000  # Frecuencia de muestreo esperada
expected_features = 240000  # El número de características que espera el modelo
genres = ['Clasica', 'Pop', 'Metal']  # Definir los géneros musicales

# Cargar el modelo preentrenado
model = joblib.load('genre_classifier_model.pkl')

# Función para descargar y convertir el audio de YouTube
def download_audio_from_youtube(url, output_file='downloaded_song.wav'):
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
                'preferredquality': '192',
            }],
            'outtmpl': 'temp_audio.%(ext)s',
        }
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # Convertir el archivo a mono y frecuencia de muestreo 48kHz
        command = [
            'ffmpeg', '-i', 'temp_audio.wav', '-ar', str(sample_rate), '-ac', '1',
            output_file, '-y'
        ]
        subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        os.remove('temp_audio.wav')  # Eliminar el archivo temporal
        return output_file
    except Exception as e:
        st.error(f"Error al descargar o procesar el audio de YouTube: {e}")
        return None

# Función para verificar que el audio sea mono y tenga la longitud adecuada
def verify_audio_shape(song):
    if len(song.shape) > 1:  # Si es estéreo
        song = song.mean(axis=1)  # Convertir a mono
    return song

# Función para extraer el espectro de Fourier de una canción cargada
def calculate_song_spectra(song, song_length):
    song_segments = []
    for _ in range(4):  # Dividimos la canción en 4 segmentos de 5 segundos
        segment_start = np.random.randint(0, song_length - segment_size)
        segment_end = segment_start + segment_size
        if segment_end > song_length:
            segment = np.zeros(segment_size)
            segment[:song_length - segment_start] = song[segment_start:]
        else:
            segment = song[segment_start:segment_end]
        spectrum = np.abs(np.fft.fft(segment))
        song_segments.append(spectrum[:segment_size])  # Usar todo el espectro
    return np.mean(song_segments, axis=0)

# Graficar la separabilidad de los géneros y la canción clasificada
def plot_genre_separability(X_train_pca, y_train, song_pca, genres):
    plt.figure(figsize=(10, 8))
    colors = sns.color_palette("viridis", len(genres))  # Colores con viridis

    for i, genre in enumerate(genres):
        plt.scatter(X_train_pca[y_train == genre, 0], X_train_pca[y_train == genre, 1],
                    color=colors[i], label=genre, alpha=0.7)

    plt.scatter(song_pca[0, 0], song_pca[0, 1], color='red', marker='x', s=200, label='Canción analizada')
    plt.title("Separabilidad de géneros y clasificación de la canción")
    plt.xlabel("PCA Componente 1")
    plt.ylabel("PCA Componente 2")
    plt.legend()
    plt.grid(True)
    st.pyplot(plt)

# Interfaz de Streamlit
st.title("Detector de Género Musical desde YouTube")

# Pedir enlace de YouTube
youtube_url = st.text_input("Ingresa el enlace de YouTube de la canción")

if youtube_url:
    # Descargar y convertir la canción de YouTube
    with st.spinner("Descargando y procesando la canción..."):
        downloaded_song_path = download_audio_from_youtube(youtube_url)

    if downloaded_song_path:
        # Leer la canción descargada
        song, sample_rate = sf.read(downloaded_song_path)

        # Verificar que la longitud del audio sea suficiente
        if len(song) >= segment_size:
            # Asegurar que la canción sea mono
            song = verify_audio_shape(song)

            # Calcular los espectros de Fourier
            spectra = calculate_song_spectra(song, len(song))

            # Verificar el tamaño del espectro (debe ser igual al esperado)
            if len(spectra) != expected_features:
                st.error(f"El espectro calculado tiene {len(spectra)} características, pero se esperaban {expected_features}.")
            else:
                # Realizar la predicción con el modelo cargado
                genre_prediction = model.predict([spectra])

                # Mostrar el resultado
                st.write(f"Género Predicho: *{genre_prediction[0]}*")

                # Reproducir 20 segundos de la canción
                segment_start = np.random.randint(0, len(song) - (20 * sample_rate))
                segment = song[segment_start:segment_start + (20 * sample_rate)]
                sf.write('segment.wav', segment, sample_rate)
                st.audio('segment.wav', format='audio/wav')
        else:
            st.error("El archivo de audio esperado es:")

In [ ]:
from pyngrok import ngrok

# Token personal de ngrok
token = "2lLFMBWiinIoZUi3bX7RF26eFeD_5KBTy8DqrMYoE2mf5Sqnc"  # Reemplaza con tu token
ngrok.set_auth_token(token)

# Ejecutar Streamlit en el puerto 5011
!nohup streamlit run app.py --server.port 5011 &

# Iniciar el túnel Ngrok y exponer Streamlit en una URL.
public_url = ngrok.connect(5011)
print(f"Aplicación aquí: {public_url}")